# Parse raw data
The original input is one nicely formatted json but we need a single json per line for every contract in order to read the data using dask.

In [ ]:
import pandas as pd
import dask.bag as db
import ijson # iterative json reading
import simplejson as json # handles Decimal type read by ijson 
from zipfile import ZipFile
import gzip
import time

base_fn = "smlouvy"
# base_fn = "smlouvy-2018-01-26"
zip_fn = f"data/hlidac/{base_fn}.zip"
data_fn = f"{base_fn}.json"

out_data_base_fn = f"data/hlidac/{base_fn}_lines"

In [ ]:
batch_size = 10000
with ZipFile(zip_fn, 'r').open(data_fn) as file_in:
    objects = ijson.items(file_in, 'item')
    i = 0
    start = time.time()
    file_out = gzip.open(f"{out_data_base_fn}_000.txt.gz", 'w')
    for o in objects:
        json_line = json.dumps(o, ensure_ascii=False, use_decimal=True)
        file_out.write((json_line + '\n').encode('utf-8'))
        i += 1
        if i%batch_size == 0:
            file_out.close()
            file_out = gzip.open(f"{out_data_base_fn}_{(i//batch_size):03}.txt.gz", 'w')
        if i%10000 == 0:
            print(f"{i}-th: {(time.time()-start)/60} min")    
    file_out.close()

Sanity check - run this only for the small data

In [ ]:
with open(f"data/hlidac/{base_fn}.json", 'r', encoding='utf-8') as file:
    data_all = json.load(file)

In [ ]:
import dask.bag as db
lines = db.read_text(f"{out_data_base_fn}_*.txt.gz")
data_all_parsed = lines.map(json.loads).compute()

In [ ]:
data_all == data_all_parsed